# Read in CSV from AWS

In [ ]:
%pyspark
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
Toys_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True)
Toys_df.show()

In [ ]:
%pyspark
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)
video_games_df.show()

# Merge DataFrames

In [ ]:
%pyspark
df_concat = video_games_df.union(Toys_df)
df_concat.count()

# Filter by voters.

In [ ]:
%pyspark
df2 = df_concat.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
df2.show(10)
df3 = df2.filter(df2['total_votes'] >= 20)
df4 = df3.filter(df3["helpful_votes"]/df3["total_votes"] >= 0.5)

# Describe stats.

In [ ]:
%pyspark
from pyspark.sql.functions import col, avg
paid_df = df4.filter(df4['vine']== 'Y')
unpaid_df = df4.filter(df4['vine']== 'N')
paid_df.describe().show()
unpaid_df.describe().show()

# Determine the percentage of five-star reviews among Vine reviews

In [ ]:
%pyspark
paid_five_star_number = paid_df[paid_df['star_rating']== 5].count()
paid_number = paid_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)
print(paid_number)
print(paid_five_star_number)
print(percentage_five_star_vine)

# Determine the percentage of five-star reviews among non-Vine reviews.

In [ ]:
%pyspark
unpaid_five_star_number = unpaid_df[unpaid_df['star_rating']== 5].count()
unpaid_number = unpaid_df.count()
percentage_five_star_non_vine = float(unpaid_five_star_number) / float(unpaid_number)
print(unpaid_number)
print(unpaid_five_star_number)
print(float(unpaid_five_star_number) / float(unpaid_number))